# **Homework solution**: Data talks club data engineering zoomcamp Data loading workshop

Course materials could be found via links:
- [homework starter](https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/cohorts/2024/workshops/dlt_resources/homework_starter.ipynb)
- [course itself ](https://github.com/DataTalksClub/data-engineering-zoomcamp/tree/main)



### Setup

In [1]:
%%capture
!pip install dlt[duckdb]

In [2]:
import dlt
import duckdb

### Task 1

Answer the following questions:

- Question 1: What is the sum of the outputs of the generator for limit = 5?
- Question 2: What is the 13th number yielded

In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

In [4]:
print(sum(square_root_generator(limit=5)))

8.382332347441762


In [5]:
for index, value in enumerate(square_root_generator(limit=13)):
    print(f'{index + 1}: {value}')

1: 1.0
2: 1.4142135623730951
3: 1.7320508075688772
4: 2.0
5: 2.23606797749979
6: 2.449489742783178
7: 2.6457513110645907
8: 2.8284271247461903
9: 3.0
10: 3.1622776601683795
11: 3.3166247903554
12: 3.4641016151377544
13: 3.605551275463989


### Task 2

Load generators to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [6]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


print('-----------------')

def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
-----------------
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [7]:
people_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_gen')

info = people_pipeline.run(
    people_1(),
    table_name='people',
    write_disposition='replace'
)

print(info)

print('------------------')

info = people_pipeline.run(
    people_2(),
    table_name='people',
    write_disposition='append'
)

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.35 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_gen
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707637510.2800767 is LOADED and contains no failed jobs
------------------
Pipeline dlt_colab_kernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_gen
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707637511.0903938 is LOADED and contains no failed jobs


In [8]:
conn = duckdb.connect(f'{people_pipeline.pipeline_name}.duckdb')
conn.sql(f"SET search_path = '{people_pipeline.dataset_name}'")

people_ages = conn.sql('SELECT SUM(age) FROM people');
display(people_ages)

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

### Task 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [9]:
info = people_pipeline.run(
    people_1(),
    table_name='people_merged',
    write_disposition='replace',
    primary_key='ID'
)

print(info)

info = people_pipeline.run(
    people_2(),
    table_name='people_merged',
    write_disposition='merge',
    primary_key='ID'
)

print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.26 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_gen
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707637511.9054585 is LOADED and contains no failed jobs
Pipeline dlt_colab_kernel_launcher load step completed in 0.38 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_gen
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1707637512.6026218 is LOADED and contains no failed jobs


In [10]:
people_count = conn.sql('SELECT COUNT(*) FROM people_merged').fetchone()
person_3_age = conn.sql('SELECT age FROM people_merged WHERE ID = 3').fetchone()

assert people_count[0] == 8
assert person_3_age[0] == 33


In [11]:
people_ages_sum = conn.sql('SELECT SUM(age) FROM people_merged')
display(people_ages_sum)

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘